# Adding EIBP Dependencies to a Slice

## Input Required Information

If you do not already have a slice created, please either create one using your own method or via the GraphML Slice Builder notebook provided. Also remember to configure your local Jupyter enviornment if you have not already (can you reach out to the nodes?).

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of the slice you wish to work on. |

In [ ]:
SLICE_NAME = "eibp"

## Access the Slice

FabOrchestrator is a custom Python class that utilizes the existing FabLib API. Its purpose is to make repetitive code easier to manage and build around features that we previously used in our custom GENI orchestrator.

The orchestrator class is initalized here, which also means the slice and its nodes are now accessable as well..

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)

except Exception as e:
    print(f"Exception: {e}")

Slice name: eibp
Slice and nodes were acquired successfully.


## Upload Configuration Script

Anything that would make sense to be configured that does not include package installation should be placed in the init_mtp.sh script. Currently, this modifies the tmux configuration after it is installed.

In [ ]:
configScript = "/home/fabric/work/FABRIC-EIBP/remote_scripts/init_mtp.sh"
manager.uploadFileParallel(configScript, prefixList="c,d,a")

File to upload: /home/fabric/work/FABRIC-EIBP/remote_scripts/init_mtp.sh
Placed in: /home/rocky/init_mtp.sh
Starting upload on node c1
Starting upload on node d1
Starting upload on node d2
Starting upload on node a1
Starting upload on node a2
Waiting for result from node c1
Output: -rw-rw-r--   1 1000     1000          292 21 Nov 17:02 ?
Waiting for result from node d1
Output: -rw-rw-r--   1 1000     1000          292 21 Nov 17:02 ?
Waiting for result from node d2
Output: -rw-rw-r--   1 1000     1000          292 21 Nov 17:02 ?
Waiting for result from node a1
Output: -rw-rw-r--   1 1000     1000          292 21 Nov 17:02 ?
Waiting for result from node a2
Output: -rw-rw-r--   1 1000     1000          292 21 Nov 17:02 ?


## Install Dependencies and Make Configuration Changes on all Nodes

Along with installing necessary pacakges, the initialization script uploaded in the prior cell is also run.

| Package | Use |
| --- | --- |
| tmux    | Terminal multiplexer similar to GNU Screen to allow for ssh sessions to disconnect and let the process continue to run. |
| Wireshark | Access to tshark, the command-line based version of the packet sniffer. |
| Development Tools | Includes all necessary applications, such as GNU GCC C compiler, make, debuggers, man pages. etc., which are needed to compile, build, and troubleshoot MTP code |

In [ ]:
# The commands to run
packagesToInstall = "sudo dnf install -q -y tmux wireshark"
devToolsInstall = 'sudo dnf groupinstall -q -y "Development Tools"'
config = 'bash init_mtp.sh {name}'

# Execute the commands
manager.executeCommandsParallel(packagesToInstall)
manager.executeCommandsParallel(devToolsInstall)
manager.executeCommandsParallel(config, prefixList="c,d,a", addNodeName=True)

Starting command on node c1
Command to execute: sudo dnf install -q -y tmux wireshark
Starting command on node d1
Command to execute: sudo dnf install -q -y tmux wireshark
Starting command on node d2
Command to execute: sudo dnf install -q -y tmux wireshark
Starting command on node a1
Command to execute: sudo dnf install -q -y tmux wireshark
Starting command on node a2
Command to execute: sudo dnf install -q -y tmux wireshark
Starting command on node ipnode-1
Command to execute: sudo dnf install -q -y tmux wireshark
Starting command on node ipnode-2
Command to execute: sudo dnf install -q -y tmux wireshark
Waiting for result from node c1
stdout: 
Installed:
  alsa-lib-1.2.8-2.el8.x86_64                                                   
  avahi-libs-0.7-20.el8.x86_64                                                  
  cups-libs-1:2.2.6-51.el8_8.2.x86_64                                           
  desktop-file-utils-0.26-1.el8.x86_64                                          
  emacs-fi

## Add Default Routes to the Compute Nodes

Sets up default routes for IP-based forwarding on compute nodes (prefix), allowing them to route traffic outside their local network.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import re

SLICE_SUPERNET = "192.168.0.0/16"

for computeNode in manager.selectedNodes("ipnode"):
    # Get the compute node's interface
    intfName = f"{computeNode.get_name()}-eth1-p1"
    intf = computeNode.get_interface(intfName)

    # Get the interface IPv4 address and its third octet
    ipAddress = intf.get_ip_addr()
    IPGroup = re.search(r"192\.168\.([0-9]{1,3})\.[0-9]{1,3}", ipAddress)
    thirdOctet = IPGroup.group(1)
    nextHop = f"192.168.{thirdOctet}.254"eth1

    # Add the route to the node
    computeNode.ip_route_add(subnet=IPv4Network(SLICE_SUPERNET), gateway=IPv4Address(nextHop))

    print(f"Adding route {SLICE_SUPERNET} to {computeNode.get_name()} with next-hop {nextHop}")

Adding route 192.168.0.0/16 to ipnode-1 with next-hop 192.168.3.254
Adding route 192.168.0.0/16 to ipnode-2 with next-hop 192.168.4.254


## Turn off Traditional IP-based Forwarding on EIBP Nodes

This is not done on compute/client nodes, as they are EIBP-unaware.

Disables traditional IP-based forwarding on EIBP nodes (prefixes) using sysctl commands.

In [ ]:
cmdOff = "sudo sysctl -w net.ipv4.ip_forward=0"
cmdOn = "sudo sysctl -w net.ipv4.ip_forward=1"
manager.executeCommandsParallel(cmdOff, prefixList="c,d,a")
manager.executeCommandsParallel(cmdOn, prefixList="ipnode")

Starting command on node c1
Command to execute: sudo sysctl -w net.ipv4.ip_forward=0
Starting command on node d1
Command to execute: sudo sysctl -w net.ipv4.ip_forward=0
Starting command on node d2
Command to execute: sudo sysctl -w net.ipv4.ip_forward=0
Starting command on node a1
Command to execute: sudo sysctl -w net.ipv4.ip_forward=0
Starting command on node a2
Command to execute: sudo sysctl -w net.ipv4.ip_forward=0
Waiting for result from node c1
stdout: net.ipv4.ip_forward = 0

stderr: 
Waiting for result from node d1
stdout: net.ipv4.ip_forward = 0

stderr: 
Waiting for result from node d2
stdout: net.ipv4.ip_forward = 0

stderr: 
Waiting for result from node a1
stdout: net.ipv4.ip_forward = 0

stderr: 
Waiting for result from node a2
stdout: net.ipv4.ip_forward = 0

stderr: 
Starting command on node ipnode-1
Command to execute: sudo sysctl -w net.ipv4.ip_forward=1
Starting command on node ipnode-2
Command to execute: sudo sysctl -w net.ipv4.ip_forward=1
Waiting for result from